### Train-on-all-but-one, decode-on-one, experiment #2

Generate decoded sample files for input to e_ F1 classifiers  
Read from normalized-within-cohort, 5k common mean absolute deviation (MAD) data file set

Demo version, created 2022-11-17,  
from b_train-on-one.ipynb

In [1]:
%whos

Interactive namespace is empty.


### Imports

In [2]:
import time

import glob as glob

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization, Activation, Layer
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# import umap.umap_ as umap - to d_

In [5]:
# target: g_devel/train-on-all_decoded, then read into the multi-experiment e_RF_DT_org-dec.ipynb

# should be renamed e_RF_DT_vae_inp-dec.ipynb

In [7]:
plt_v=17 # Train-on-all, experiment 4, using normalize within cohort data: g_devel/data_GEXP_5k_col_norm

### Build Model

In [4]:
# Function set for leave one out loop
def compute_latent(x): # x:
    mu, sigma = x
    batch = K.shape(mu)[0]
    dim = K.shape(mu)[1]
    eps = K.random_normal(shape=(batch,dim), mean=0., stddev=1.0 )
    return mu + K.exp(sigma/2)*eps

In [5]:
# Set custom layer , will need all this at start of shell script
class CustomVariationalLayer(Layer):
    """
    Define a custom layer
    """
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [6]:
class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

In [ ]:
# guess we need to restack the frames and hold one out
# start with getting a cohort list

In [12]:
nrm_wthn_pths

['data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/COADREAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/OV_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PAAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv',
 'da

In [14]:
len(nrm_wthn_pths)

25

In [22]:
full_stack = pd.DataFrame() # Production cell 1, stack up the cancers
for path in nrm_wthn_pths:
    print(path)
    train_df = pd.read_csv(path, sep="\t", index_col=0)
    full_stack = pd.concat( [full_stack, train_df], axis = 0 )
    # break

data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/COADREAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/OV_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PAAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SKCM_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/TGCT_gexp_

In [23]:
full_stack

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,ACC_2,0.000182,0.360981,0.001892,0.007364,0.000000,0.000000,0.000000,0.000284,0.006090,...,0.246175,0.542221,0.402793,0.335923,0.241655,0.232638,0.137530,0.278195,0.306376,0.078349
TCGA-OR-A5J2,ACC_2,0.000662,0.028076,0.016630,0.094906,0.001782,0.002093,0.000000,0.052871,0.002012,...,0.057344,0.810534,0.198127,0.339276,1.000000,0.255464,0.184041,0.413199,0.303635,0.039638
TCGA-OR-A5J3,ACC_3,0.000675,0.504978,0.004538,0.036593,0.002623,0.001027,0.000000,0.002378,0.000851,...,0.457210,0.488562,0.499732,0.410338,0.493998,0.576924,0.231120,0.519779,0.540690,0.028952
TCGA-OR-A5J5,ACC_3,0.039602,0.519118,0.005965,0.081847,0.000000,0.001342,0.000000,0.000142,0.001235,...,0.145073,0.509690,0.383014,0.394384,0.254431,0.305898,0.222402,0.371465,0.304460,0.073477
TCGA-OR-A5J6,ACC_1,0.000141,0.017320,0.001465,0.003721,0.000000,0.000000,0.000000,0.007885,0.002986,...,0.083668,0.411400,0.346296,0.393668,0.321142,0.231314,0.780143,0.318338,0.992842,0.742167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WC-AA9E,UVM_3,0.030804,0.005387,0.003946,0.020549,0.102395,0.082625,0.076004,0.019509,0.055003,...,0.393935,0.610252,0.344880,0.686383,0.853623,0.598790,0.528979,0.337590,0.276482,0.037362
TCGA-YZ-A980,UVM_1,0.028575,0.077464,0.094785,0.145219,0.004130,0.000684,0.001621,0.000118,0.313738,...,0.448846,0.425112,0.702758,0.458945,0.515307,0.449282,0.272083,0.430847,0.303070,0.049678
TCGA-YZ-A982,UVM_2,0.000000,0.005380,0.007426,0.031989,0.023712,0.011788,0.000000,0.000149,0.295075,...,0.551889,0.250576,0.607254,0.601735,0.518152,0.364811,0.395582,0.302280,0.306822,0.009950
TCGA-YZ-A983,UVM_3,0.021407,0.475421,0.025232,0.039406,0.179425,0.042036,0.159049,0.000309,0.082759,...,0.341519,0.404413,0.578043,0.483949,0.766207,0.827500,0.300552,0.386852,0.201858,0.024924


In [24]:
full_stack.Labels.str.split('_').str[0].unique()

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [27]:
c_lst = full_stack.Labels.str.split('_').str[0].unique() # Production cell 2

In [25]:
full_stack.Labels.str.contains('ACC') # gives the boolean

TCGA-OR-A5J1     True
TCGA-OR-A5J2     True
TCGA-OR-A5J3     True
TCGA-OR-A5J5     True
TCGA-OR-A5J6     True
                ...  
TCGA-WC-AA9E    False
TCGA-YZ-A980    False
TCGA-YZ-A982    False
TCGA-YZ-A983    False
TCGA-YZ-A985    False
Name: Labels, Length: 8009, dtype: bool

In [26]:
~full_stack.Labels.str.contains('ACC') # tilda is friend

TCGA-OR-A5J1    False
TCGA-OR-A5J2    False
TCGA-OR-A5J3    False
TCGA-OR-A5J5    False
TCGA-OR-A5J6    False
                ...  
TCGA-WC-AA9E     True
TCGA-YZ-A980     True
TCGA-YZ-A982     True
TCGA-YZ-A983     True
TCGA-YZ-A985     True
Name: Labels, Length: 8009, dtype: bool

In [28]:
for c in c_lst:
    stack_less_one = full_stack[~full_stack.Labels.str.contains(c)]
    break

In [30]:
8009-7933 # sample count for adrenocortical, check

76

In [40]:
c_lst

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [41]:
c_lst = ['ACC', 'BLCA']

In [56]:
# Train-on-one using within-cohort normed data
# for c in stacked.Labels.str.split('_').str[0].unique():
c_lst = full_stack.Labels.str.split('_').str[0].unique()
# c_lst = ['ACC', 'BLCA'] # Devel cohort control
plt_v = 17
nrm_wthn_pths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))   
for c in c_lst:
    stack_less_one = full_stack[~full_stack.Labels.str.contains(c)]
    decode_df = full_stack[full_stack.Labels.str.contains(c)]
    # break
    features = stack_less_one.columns[1:]
    
    original_dim = len(features)
    feature_dim = len(features)
    latent_dim = 100
    
    encoder_inputs = keras.Input(shape=(feature_dim,))
    z_mean_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_1")(encoder_inputs)
    z_mean_dense_batchnorm = layers.BatchNormalization()(z_mean_dense_linear)
    z_mean_encoded = layers.Activation('relu')(z_mean_dense_batchnorm)

    z_log_var_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_2")(encoder_inputs)
    z_log_var_dense_batchnorm = layers.BatchNormalization()(z_log_var_dense_linear)
    z_log_var_encoded = layers.Activation('relu')(z_log_var_dense_batchnorm)

    latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,), name="latent_space")([z_mean_encoded, z_log_var_encoded])

    decoder_to_reconstruct = layers.Dense(feature_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
    decoder_outputs = decoder_to_reconstruct(latent_space)

    learning_rate = 0.0005
    kappa = 1

    beta = K.variable(0)

    adam = optimizers.Adam(learning_rate=learning_rate)
    vae_layer = CustomVariationalLayer()([encoder_inputs, decoder_outputs])
    vae = Model(encoder_inputs, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

    epochs=35

    fit_start = time.time()
    history = vae.fit(stack_less_one.iloc[:, 1:],  #       <-------- fit the VAE here
                epochs=epochs, batch_size=50, shuffle=True,
                callbacks=[WarmUpCallback(beta, kappa)],
                     # verbose=0 #  <---------------------- turn this on for production
                     )
    fit_end = time.time() - fit_start

    plt.plot(history.history['loss'],label="loss")

    plt.title(
        c+' held out from fitting'+
        '\nTybalt VAE train\n'
             )
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.annotate('Feature set = '+'common MAD 5000k'+
                 '\nLatent dim = '+str(latent_dim)+
                 '\nLayer type = dense'+
                 '\nNormalization = divide by max(),\n'+
                 '                       column-wise,\n'+
                 '                       within each cohort'+
                 '\nFit time = '+str(round(fit_end,2 )),
                xy=(.4, .8), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                )

    plt.legend(loc="lower left")
    plt.savefig(
        'train-on-all_loss_plots/'+c+'_held_out_'+
        '_fit_on_n=24_cohorts_'+str(plt_v)+'.png', bbox_inches='tight') # version 17, norm within cohort
    plt.close()
        
    encoder = Model(encoder_inputs, z_mean_encoded)
    decoder_input = keras.Input(shape=(latent_dim, ))
    _x_decoded_mean = decoder_to_reconstruct(decoder_input)
    decoder = Model(decoder_input, _x_decoded_mean)

    y_df = decode_df.Labels

    decoded = pd.DataFrame(decoder.predict(encoder.predict(decode_df.iloc[:, 1:])),
                           index=decode_df.index, columns=decode_df.iloc[:, 1:].columns)
    
    latent_object = pd.DataFrame(encoder.predict(decode_df.iloc[:, 1:]),
                 index=decode_df.index,
                    # columns=decode_df.iloc[:, 1:].columns # Columns are different because this is latent space
            )
    latent_object.index.name = c
    latent_object.to_csv('train-on-all_latent_objects/trn-on-n24_'+c+'_latent_object.tsv',
        sep = '\t')
    # break
    decoded_labeled = pd.concat( [pd.DataFrame(y_df), decoded] , axis =1)
    decoded_labeled.to_csv(
        'train-on-all_decoded/trn-on-n24_'+c+'_decoded,'+
        str(round(fit_end,2))+'_fit_time.tsv',
        sep = '\t')

    print(str(c) + ' done')
    print(' ')
    # break
print('Train-on-all experiment done')

2022-11-17 13:42:38.615208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 13:42:38.686995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 13:42:38.726154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Train on 7933 samples
Epoch 1/35


2022-11-17 13:42:38.988336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 13:42:39.071863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 13:42:39.106699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7933/7933 [==============================] - 3s 360us/sample - loss: 2702.9409
Epoch 2/35
 650/7933 [=>............................] - ETA: 1s - loss: 2457.5661

2022-11-17 13:42:41.743313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

7935/7935 [==============================] - 4s 566us/sample - loss: 2698.9256
Epoch 2/35
 450/7935 [>.............................] - ETA: 2s - loss: 2457.7089

2022-11-17 14:13:58.943202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7935/7935 [==============================] - 2s 287us/sample - loss: 2416.0437
Epoch 3/35
7935/7935 [==============================] - 2s 283us/sample - loss: 2372.7990
Epoch 4/35
7935/7935 [==============================] - 2s 281us/sample - loss: 2346.5676
Epoch 5/35
7935/7935 [==============================] - 2s 279us/sample - loss: 2329.3198
Epoch 6/35
7935/7935 [==============================] - 2s 282us/sample - loss: 2316.7505
Epoch 7/35
7935/7935 [==============================] - 2s 283us/sample - loss: 2307.8741
Epoch 8/35
7935/7935 [==============================] - 2s 280us/sample - loss: 2301.0893
Epoch 9/35
7935/7935 [==============================] - 2s 286us/sample - loss: 2295.8365
Epoch 10/35
7935/7935 [==============================] - 2s 281us/sample - loss: 2291.6392
Epoch 11/35
7935/7935 [==============================] - 2s 287us/sample - loss: 2287.7995
Epoch 12/35
7935/7935 [==============================] - 2s 283us/sample - loss: 2284.7776
Epoch 13/35
7935/7

/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-17 14:15:15.705817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 14:15:16.168387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


UVM done
 
Train-on-all experiment done


In [54]:
latent_object.index.name = c

In [55]:
latent_object

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
BLCA,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO,0.000000,0.000000,0.606011,1.346400,0.912834,0.801650,0.000000,0.000000,0.042244,1.362930,...,1.296238,0.006343,0.421620,1.418529,0.787730,0.112657,0.663817,0.119403,0.212188,1.025270
TCGA-2F-A9KP,0.288308,0.225680,1.369809,1.509376,0.805904,0.040974,0.000000,0.000000,0.578807,0.038450,...,0.838576,0.086969,1.046012,0.640437,0.017200,0.000000,0.805915,0.216522,0.541130,0.743966
TCGA-2F-A9KQ,0.174633,0.113791,0.500337,1.979563,0.604503,0.557613,0.000000,0.257109,0.785105,0.438577,...,0.824561,0.000000,1.326405,0.762011,0.524845,0.401829,0.781440,0.202172,0.163461,0.468314
TCGA-2F-A9KR,0.389048,0.000000,0.000000,2.456492,0.667225,0.878729,0.000000,0.432306,1.628833,0.305893,...,1.777221,0.000000,0.462008,1.360465,0.411681,0.000000,1.109046,0.634260,0.341119,0.884869
TCGA-2F-A9KT,2.024923,0.000000,1.488242,1.318837,1.024441,1.242546,0.000000,1.081394,1.904202,0.000000,...,1.413479,0.322844,0.348878,0.065319,0.000000,0.000000,0.992222,0.409808,0.731071,0.585544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,0.076250,0.000000,0.939976,2.171430,1.281768,0.914050,0.000000,0.000000,0.920151,0.000000,...,0.615883,0.000000,0.909881,0.824910,0.022802,0.000000,1.874989,0.050035,1.136423,0.450029
TCGA-ZF-AA58,0.101546,0.000000,1.548029,0.753259,0.669541,0.000000,2.219252,0.719016,0.000000,1.374755,...,0.013495,0.000000,1.067261,0.000000,0.000000,1.216576,0.411895,0.000000,0.156087,0.000000
TCGA-ZF-AA5H,1.541133,1.062936,2.174954,1.016264,0.912118,0.803701,2.345641,1.474476,0.166684,0.000000,...,0.000000,0.000000,1.174279,0.000000,0.000000,1.799553,1.559334,0.334298,0.159321,0.000000


In [ ]:
# Check decoded and latent files

In [44]:
path = '/Users/karlberb/Documents/work/sample_count/synthetic_TCGA_data_gen/g_devel/train-on-all_decoded/trn-on-n24_ACC_decoded32.23_fit_time.tsv'

In [45]:
check_decoded = pd.read_csv(path, sep="\t", index_col=0)

In [46]:
check_decoded

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,ACC_2,0.073418,0.147307,0.031702,0.038209,0.020634,0.019491,0.010490,0.025941,0.078065,...,0.574548,0.389574,0.573389,0.544395,0.360301,0.504379,0.423794,0.447221,0.398944,0.227983
TCGA-OR-A5J2,ACC_2,0.030036,0.191190,0.127843,0.242458,0.051320,0.042658,0.051765,0.045808,0.148640,...,0.230981,0.529943,0.484048,0.571946,0.695771,0.510007,0.239454,0.531904,0.496191,0.131224
TCGA-OR-A5J3,ACC_3,0.101574,0.190120,0.133091,0.175249,0.068187,0.073265,0.080703,0.050058,0.206919,...,0.448891,0.301544,0.547895,0.524885,0.425077,0.590461,0.379475,0.595780,0.492072,0.233061
TCGA-OR-A5J5,ACC_3,0.084666,0.229070,0.125040,0.083362,0.055047,0.052270,0.022353,0.035620,0.044453,...,0.563271,0.402082,0.420828,0.465709,0.420051,0.323115,0.292053,0.473378,0.516511,0.230934
TCGA-OR-A5J6,ACC_1,0.180645,0.185286,0.051193,0.104497,0.036366,0.043263,0.033034,0.114559,0.342766,...,0.302504,0.256770,0.453744,0.372566,0.359545,0.461527,0.588254,0.413971,0.380170,0.355213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,ACC_3,0.066576,0.245109,0.052169,0.123841,0.053968,0.047913,0.043481,0.026307,0.045134,...,0.309225,0.453690,0.573917,0.585920,0.590135,0.550293,0.317906,0.330269,0.468219,0.192450
TCGA-PA-A5YG,ACC_1,0.087232,0.246409,0.092916,0.194300,0.090525,0.074725,0.091646,0.103958,0.186069,...,0.230191,0.390657,0.627422,0.470588,0.510122,0.591726,0.451206,0.492860,0.484135,0.326528
TCGA-PK-A5H9,ACC_1,0.147418,0.133628,0.043239,0.104964,0.015833,0.018731,0.016373,0.041694,0.169856,...,0.236903,0.428183,0.434133,0.491068,0.517655,0.339661,0.391444,0.229774,0.283653,0.271234
TCGA-PK-A5HA,ACC_1,0.016393,0.106773,0.089838,0.190512,0.038944,0.037263,0.042446,0.111982,0.197803,...,0.212212,0.423918,0.447059,0.412843,0.499841,0.439405,0.315867,0.452091,0.464136,0.230124


In [52]:
path = 'train-on-all_latent_objects/trn-on-n24_BLCA_latent_object.tsv'

In [53]:
check_latent = pd.read_csv(path, sep="\t", index_col=0)
check_latent

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
TCGA-2F-A9KO,0.000000,0.000000,0.606011,1.346400,0.912834,0.801650,0.000000,0.000000,0.042244,1.362930,...,1.296238,0.006343,0.421620,1.418529,0.787730,0.112657,0.663817,0.119403,0.212188,1.025270
TCGA-2F-A9KP,0.288308,0.225680,1.369809,1.509376,0.805904,0.040974,0.000000,0.000000,0.578807,0.038450,...,0.838576,0.086969,1.046012,0.640437,0.017200,0.000000,0.805915,0.216522,0.541130,0.743966
TCGA-2F-A9KQ,0.174633,0.113791,0.500337,1.979564,0.604503,0.557613,0.000000,0.257109,0.785105,0.438577,...,0.824561,0.000000,1.326405,0.762011,0.524845,0.401829,0.781440,0.202172,0.163461,0.468314
TCGA-2F-A9KR,0.389048,0.000000,0.000000,2.456492,0.667225,0.878729,0.000000,0.432306,1.628833,0.305893,...,1.777221,0.000000,0.462008,1.360465,0.411681,0.000000,1.109046,0.634260,0.341119,0.884869
TCGA-2F-A9KT,2.024923,0.000000,1.488242,1.318837,1.024441,1.242546,0.000000,1.081394,1.904202,0.000000,...,1.413479,0.322844,0.348878,0.065319,0.000000,0.000000,0.992222,0.409808,0.731071,0.585544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,0.076250,0.000000,0.939976,2.171430,1.281768,0.914050,0.000000,0.000000,0.920151,0.000000,...,0.615883,0.000000,0.909881,0.824910,0.022802,0.000000,1.874989,0.050035,1.136423,0.450029
TCGA-ZF-AA58,0.101546,0.000000,1.548029,0.753259,0.669541,0.000000,2.219252,0.719016,0.000000,1.374755,...,0.013495,0.000000,1.067261,0.000000,0.000000,1.216576,0.411895,0.000000,0.156087,0.000000
TCGA-ZF-AA5H,1.541133,1.062936,2.174954,1.016264,0.912118,0.803701,2.345641,1.474476,0.166684,0.000000,...,0.000000,0.000000,1.174279,0.000000,0.000000,1.799553,1.559334,0.334298,0.159321,0.000000
TCGA-ZF-AA5N,0.807208,0.000000,2.474444,1.837770,0.869297,0.104512,0.568800,0.788111,0.000000,0.433905,...,0.551629,0.000000,0.918376,0.000000,0.140257,0.223592,1.860273,0.000000,0.544417,0.000000
